# Detailed timing of the lensing operator construction 

In [1]:
import numpy as np
from scipy import sparse
import time

import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
import lenstronomy.Util.util as lenstro_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.LensModel.lens_model import LensModel

from slitronomy.Lensing.lensing_operator import LensingOperator, LensingOperatorInterpol 

In [2]:
class Timer(object):
    """simple class for easy timing"""
    def __init__(self):
        pass
    @classmethod
    def start(cls, txt):
        cls._txt = txt
        cls._time = time.time()
        cls._total = 0
    @classmethod
    def stop(cls):
        print("time for {} : {:.3e} s".format(cls._txt, time.time()-cls._time))
        cls._total += time.time()-cls._time
        del cls._time
        del cls._txt
    @classmethod
    def start_loop(cls, txt, ID):
        """To be used when timing a routine that is executed several times in a loop (assigning unique ID)"""
        if not hasattr(cls, '_loop_ids'):
            cls._loop_ids = {}
            cls._loop_txts = {}
        if not ID in cls._loop_ids:
            cls._loop_ids[ID] = []  # create new ID (i.e. new set of timings for a specific loop)
        cls._loop_txts[ID] = txt
        cls._loop_ids[ID].append(time.time())  # add start time
    @classmethod
    def stop_loop(cls, ID):
        """stop timer for last item if the list corresponding to this ID"""
        cls._loop_ids[ID][-1] = time.time() - cls._loop_ids[ID][-1]
    @classmethod
    def print_loop(cls, ID):
        """Print detailed timing string for this ID"""
        mean = np.mean(cls._loop_ids[ID])
        std = np.std(cls._loop_ids[ID])
        print("average time for {} : {:.3e} ± {:.3e} s".format(cls._loop_txts[ID], mean, std))
        cls._total += mean
        del cls._loop_ids[ID]
        del cls._loop_txts[ID]
    @classmethod
    def print_total(cls):
        print("total : {} s".format(cls._total))
        del cls._total

In [3]:
num_pix = 99
delta_pix = 0.08

_, _, ra_at_xy_0, dec_at_xy_0, _, _, Mpix2coord, _ \
    = lenstro_util.make_grid_with_coordtransform(numPix=num_pix, deltapix=delta_pix, subgrid_res=1, 
                                                 inverse=False, left_lower=False)
kwargs_data = {
    #'background_rms': background_rms,
    #'exposure_time': np.ones((num_pix, num_pix)) * exp_time,  # individual exposure time/weight per pixel
    'ra_at_xy_0': ra_at_xy_0, 'dec_at_xy_0': dec_at_xy_0, 
    'transform_pix2angle': Mpix2coord,
    'image_data': np.zeros((num_pix, num_pix))
}
data_class = ImageData(**kwargs_data)

lens_model_list = ['SPEP']
kwargs_spemd = {'theta_E': 2, 'gamma': 2, 'center_x': 0, 'center_y': 0, 'e1': -0.05, 'e2': 0.05}
kwargs_lens = [kwargs_spemd]#, kwargs_shear]
lens_model_class = LensModel(lens_model_list=lens_model_list)

kwargs_special = {'delta_x_source_grid': 0, 'delta_y_source_grid': 0}

In [4]:
# init the 'simple' operator
lensing_op = LensingOperator(data_class, lens_model_class, subgrid_res_source=1, 
                             likelihood_mask=None, minimal_source_plane=False, min_num_pix_source=10,
                             fix_minimal_source_plane=True, matrix_prod=True)

# init the 'interpol' operator
lensing_op_interp = LensingOperatorInterpol(data_class, lens_model_class, subgrid_res_source=1, 
                             likelihood_mask=None, minimal_source_plane=False, min_num_pix_source=10,
                             fix_minimal_source_plane=True)

### Copy-paste of methods from `slitronomy` for detailed timing

In [5]:
def compute_mapping_TIMING(lensing_operator_class, kwargs_lens, kwargs_special):
    self_ = lensing_operator_class  # alias
    
    # delete previous mapping and init the new one
    self_.delete_mapping()

    # initialize matrix
    Timer.start("mapping init")
    if self_._matrix_prod:
        lens_mapping_matrix = np.zeros((self_.imagePlane.grid_size, self_.sourcePlane.grid_size))
    else:
        lens_mapping_list = []
    Timer.stop()

    # backward ray-tracing to get source coordinates in image plane (the 'betas')
    Timer.start("ray shooting")
    beta_x, beta_y = self_.lensModel.ray_shooting(self_.imagePlane.theta_x, self_.imagePlane.theta_y, kwargs_lens)
    Timer.stop()
    
    # get source plane offsets
    grid_offset_x, grid_offset_y = self_._source_grid_offsets(kwargs_special)

    # iterate through indices of image plane (indices 'i')
    Timer.start("pixel iteration")
    for i in range(self_.imagePlane.grid_size):
        # find source pixel that corresponds to ray traced image pixel
        Timer.start_loop("find source pixel", ID=0)
        j = self_._find_source_pixel(i, beta_x, beta_y, grid_offset_x=grid_offset_x, grid_offset_y=grid_offset_y)
        Timer.stop_loop(ID=0)
        
        # fill the mapping array
        Timer.start_loop("filling array", ID=1)
        if self_._matrix_prod:
            lens_mapping_matrix[i, j] = 1
        else:
            lens_mapping_list.append(j)
        Timer.stop_loop(ID=1)
    Timer.stop()
    Timer.print_loop(ID=0)
    Timer.print_loop(ID=1)

    if self_._matrix_prod:
        # convert numpy array to sparse matrix, using Compressed Sparse Row (CSR) format for fast vector products
        Timer.start("convert matrix to sparse matrix")
        self_._lens_mapping = sparse.csr_matrix(lens_mapping_matrix)
        Timer.stop()
    else:
        # convert the list to array
        Timer.start("convert list to array")
        self_._lens_mapping = np.array(lens_mapping_list)
        Timer.stop()
        
    Timer.print_total()
        

def compute_mapping_interpol_TIMING(lensing_operator_interpol_class, kwargs_lens, kwargs_special):
    self_ = lensing_operator_interpol_class  # alias
    
    # delete previous mapping and init the new one
    self_.delete_mapping()

    # initialize matrix
    Timer.start("mapping init")
    lens_mapping_matrix = np.zeros((self_.imagePlane.grid_size, self_.sourcePlane.grid_size))
    Timer.stop()

    # backward ray-tracing to get source coordinates in image plane (the 'betas')
    Timer.start("ray shooting")
    beta_x, beta_y = self_.lensModel.ray_shooting(self_.imagePlane.theta_x, self_.imagePlane.theta_y, kwargs_lens)
    Timer.stop()
    
    # get source plane offsets
    grid_offset_x, grid_offset_y = self_._source_grid_offsets(kwargs_special)
    
    # iterate through indices of image plane (indices 'i')
    Timer.start("pixel iteration")
    for i in range(self_.imagePlane.grid_size):
        # find source pixel that corresponds to ray traced image pixel
        Timer.start_loop("finding neighbouring pixels", ID=0)
        j_list, _, dist_A_x, dist_A_y = self_._find_surrounding_source_pixels(i, beta_x, beta_y, grid_offset_x=grid_offset_x, grid_offset_y=grid_offset_y)
        Timer.stop_loop(ID=0)
        
        # get interpolation weights
        weight_list = self_._bilinear_weights(dist_A_x, dist_A_y)

        # remove pixels and weights that are outside source plane grid
        if self_._minimal_source_plane:
            Timer.start_loop("checking inside grid", ID=1)
            j_list, weight_list = self_._check_inside_grid(j_list, weight_list)
            Timer.stop_loop(ID=1)

        # fill the mapping arrays
        Timer.start_loop("filling array", ID=2)
        lens_mapping_matrix[i, j_list] = weight_list
        Timer.stop_loop(ID=2)
        
    Timer.stop()
    Timer.print_loop(ID=0)
    if self_._minimal_source_plane:
        Timer.print_loop(ID=1)
    Timer.print_loop(ID=2)
    
    # convert numpy array to sparse matrix, using Compressed Sparse Row (CSR) format for fast vector products
    Timer.start("convert matrix to sparse matrix")
    self_._lens_mapping = sparse.csr_matrix(lens_mapping_matrix)
    Timer.stop()
    
    Timer.print_total()

## Detailed timing for `source_subgrid_res = 1`

In [6]:
lensing_op.sourcePlane.subgrid_resolution = 1
lensing_op_interp.sourcePlane.subgrid_resolution = 1

In [7]:
%timeit lensing_op._compute_mapping(kwargs_lens, kwargs_special)
%timeit lensing_op.update_mapping(kwargs_lens, kwargs_special)

%timeit lensing_op_interp._compute_mapping(kwargs_lens, kwargs_special)
%timeit lensing_op_interp.update_mapping(kwargs_lens, kwargs_special)

# update_mapping adds little overhead compared to _compute_mapping due to computing source mask

1.26 s ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.22 s ± 8.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.73 s ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.68 s ± 48.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
compute_mapping_TIMING(lensing_op, kwargs_lens, kwargs_special)

time for mapping init : 2.098e-05 s
time for ray shooting : 1.692e-03 s
time for pixel iteration : 4.430e-01 s
average time for find source pixel : 4.134e-05 ± 9.423e-06 s
average time for filling array : 2.390e-06 ± 1.374e-06 s
time for convert matrix to sparse matrix : 7.550e-01 s
total : 0.7551021575927734 s


In [9]:
compute_mapping_interpol_TIMING(lensing_op_interp, kwargs_lens, kwargs_special)

time for mapping init : 2.003e-05 s
time for ray shooting : 1.703e-03 s
time for pixel iteration : 8.565e-01 s
average time for finding neighbouring pixels : 7.802e-05 ± 1.477e-05 s
average time for filling array : 6.511e-06 ± 2.545e-06 s
time for convert matrix to sparse matrix : 7.520e-01 s
total : 0.7520828247070312 s


## Comparison when increasing `source_subgrid_res`

In [10]:
lensing_op.sourcePlane.subgrid_resolution = 2
lensing_op_interp.sourcePlane.subgrid_resolution = 2

In [ ]:
%timeit lensing_op.update_mapping(kwargs_lens, kwargs_special)
%timeit lensing_op_interp.update_mapping(kwargs_lens, kwargs_special)

4.47 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
compute_mapping_TIMING(lensing_op, kwargs_lens, kwargs_special)

In [ ]:
compute_mapping_interpol_TIMING(lensing_op_interp, kwargs_lens, kwargs_special)

### Converting a dense matrix to a sparse matrix

In [ ]:
n1, n2 = 99**2, 99**2
#n1, n2 = 10, 10
%timeit np.random.rand(n1, n2)

In [ ]:
dense_mat = np.random.rand(n1, n2)
print(dense_mat.shape)

In [ ]:
from scipy import sparse

In [ ]:
%timeit sparse.csr_matrix(dense_mat)

In [ ]:
sparse_mat = sparse.csr_matrix(dense_mat)

In [ ]:
sparse_mat

In [ ]:
%timeit np.squeeze(np.maximum(1, dense_mat.sum(axis=0)))
%timeit np.squeeze(np.maximum(1, sparse_mat.sum(axis=0)).A)

In [ ]:
norm_from_dense = np.squeeze(np.maximum(1, dense_mat.sum(axis=0)))
print(norm_from_dense.shape)

norm_from_sparse = np.squeeze(np.maximum(1, sparse_mat.sum(axis=0)).A)
print(norm_from_sparse.shape)